<h1>Model - Number of Trips<h1>

In [ ]:
import tensorflow

In [ ]:
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from pyspark.sql import SparkSession
import pandas as pd

In [5]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/09 11:35:38 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.12.229.205 instead (on interface en0)
22/08/09 11:35:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/09 11:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/09 11:35:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/09 11:35:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/09 11:35:39 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/09 11:35:39 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
df = pd.read_parquet('../data/curated/combined_data')

In [12]:
df.show(1, vertical = True)

-RECORD 0--------------------
 PU_Location_ID | 25         
 DO_Location_ID | 62         
 Driver_pay     | 8.48       
 Trip_time      | 660        
 Date           | 2019-03-01 
 Temperature_C  | -1.11      
 Humidity_%     | 47.0       
 Speed_kmh      | 15.29      
 Precip_Rate_mm | 0.0        
 Pressure_hPa   | 1025.74    
 Hour           | 0          
 Day_of_week    | 6          
only showing top 1 row



Make sure PU_Location_ID isn't being treated as a number (ordinal)

In [13]:
features = 'features'
input_cols = ['PU_Location_ID', 'Temperature_C', 'Humidity_%', 'Speed_kmh', 'Precip_Rate_mm', 'Hour', 'Day_of_week']

assembler = VectorAssembler(
    # which column to combine
    inputCols=input_cols, 
    # How should the combined columns be named
    outputCol=features
)

model_sdf = assembler.transform(df.dropna('any'))

In [15]:
model_sdf.select('features').head(5), model_sdf.select('Driver_pay').head(5)

([Row(features=DenseVector([25.0, -1.11, 47.0, 15.29, 0.0, 0.0, 6.0])),
  Row(features=DenseVector([65.0, -1.11, 47.0, 15.29, 0.0, 0.0, 6.0])),
  Row(features=DenseVector([140.0, -1.11, 47.0, 15.29, 0.0, 0.0, 6.0])),
  Row(features=DenseVector([161.0, -1.11, 47.0, 15.29, 0.0, 0.0, 6.0])),
  Row(features=DenseVector([209.0, -1.11, 47.0, 15.29, 0.0, 0.0, 6.0]))],
 [Row(Driver_pay=8.48),
  Row(Driver_pay=20.62),
  Row(Driver_pay=19.49),
  Row(Driver_pay=34.71),
  Row(Driver_pay=13.02)])

In [16]:
splits = df.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

In [18]:
data = spark.read.format("libsvm")\
        .load("data/mllib/sample_multiclass_classification_data.txt")

AnalysisException: Path does not exist: file:/Users/senturner/Documents/GitHub/mast30034-project-1-senturner/notebooks/data/mllib/sample_multiclass_classification_data.txt

In [17]:
layers = [4, 5, 4, 3]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

IllegalArgumentException: features does not exist. Available: PU_Location_ID, DO_Location_ID, Driver_pay, Trip_time, Date, Temperature_C, Humidity_%, Speed_kmh, Precip_Rate_mm, Pressure_hPa, Hour, Day_of_week